In [18]:
import pandas as pd

# Load dataset
df = pd.read_csv(r'C:\Users\Emmanuel\Desktop\Data Science Project\Food-Delivery (1).csv', encoding='latin1')

# Convert to timedelta
df['order_time'] = pd.to_timedelta(df['Time customer placed order'].astype(str), errors='coerce')
df['delivery_time'] = pd.to_timedelta(df['Delivery time'].astype(str), errors='coerce')

# Compute delivery duration in minutes
df['delivery_duration_mins'] = (df['delivery_time'] - df['order_time']).dt.total_seconds() / 60
df.loc[df['delivery_duration_mins'] < 0, 'delivery_duration_mins'] += 1440  # handle overnight wrap-around

# Format order_time and delivery_time to HH:MM
df['order_time'] = df['order_time'].apply(lambda x: f"{int(x.total_seconds() // 3600):02}:{int((x.total_seconds() % 3600) // 60):02}")
df['delivery_time'] = df['delivery_time'].apply(lambda x: f"{int(x.total_seconds() // 3600):02}:{int((x.total_seconds() % 3600) // 60):02}")

# Format delivery duration as HH:MM
df['delivery_duration'] = df['delivery_duration_mins'].apply(lambda x: f"{int(x // 60):02}:{int(x % 60):02}")

# Classify delivery speed
def classify_delivery_speed(mins):
    if mins <= 30:
        return 'Fast'
    elif mins <= 60:
        return 'Moderate'
    elif mins <= 90:
        return 'Slow'
    else:
        return 'Very Slow'

df['delivery_rating'] = df['delivery_duration_mins'].apply(lambda x: classify_delivery_speed(int(x)))

# Drop raw duration in minutes if not needed
df.drop(columns=['delivery_duration_mins'], inplace=True)

# Save to final file
df.to_csv("Food Delivery Final Format.csv", index=False)

# Optional: Preview result
print(df[['Delivery Area', 'order_time', 'delivery_time', 'delivery_duration', 'delivery_rating']].head())


  Delivery Area order_time delivery_time delivery_duration delivery_rating
0       Fremont      02:52         03:35             00:43        Moderate
1       Hayward      15:58         18:57             02:58       Very Slow
2       Hayward      19:02         19:31             00:28            Fast
3    Union City      04:01         04:37             00:36        Moderate
4       Hayward      23:46         00:37             00:50        Moderate
